In [ ]:
# Importar librerías
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from numpy.core.numeric import NaN

# Lectura del dataset
directorio_dataset = '/content/drive/MyDrive/TFM Big Data & BA/Propuestas/Proyecto Final/0 - Dataset/train_data.csv'
df = pd.read_csv(directorio_dataset)

# Selección de variables
df = df[['price', 'category', 'subcategory', 'region', 'sold']]
df.region = df.region.astype("category")
df.category = df.category.astype("category")
df.subcategory = df.subcategory.astype("category")
df.sold = df.sold.astype("category")

# Función percentil
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

# Según la categoría, la subcategoría y la región del producto
df_p = df.groupby(['category', 'subcategory', 'region', 'sold']).agg([np.mean, 'min', 'max', 
                                                                      percentile(25), percentile(30), percentile(40),
                                                                      percentile(60), percentile(70), percentile(75)]).reset_index()
df_p_2 = df_p[df_p['price', 'mean'].notnull()]
df_p_2 = df_p_2[df_p_2.sold == 1]

# Comprobamos que no existen nulos ni NaNs
print("-------------------------------------------------------------")
print('Registros NULL en precios por categoría/subcategoría/región:')
print(df_p_2.isnull().sum())
print("-------------------------------------------------------------")
print('Registros NAN en precios por categoría/subcategoría/región:')
print(df_p_2.isna().sum())

Mounted at /content/drive
-------------------------------------------------------------
Registros NULL en precios por categoría/subcategoría/región:
category                      0
subcategory                   0
region                        0
sold                          0
price        mean             0
             min              0
             max              0
             percentile_25    0
             percentile_30    0
             percentile_40    0
             percentile_60    0
             percentile_70    0
             percentile_75    0
dtype: int64
-------------------------------------------------------------
Registros NAN en precios por categoría/subcategoría/región:
category                      0
subcategory                   0
region                        0
sold                          0
price        mean             0
             min              0
             max              0
             percentile_25    0
             percentile_30    0
            

In [ ]:
df_p_2['diferencia'] = (df_p_2.price.percentile_60 - df_p_2.price.percentile_40)

In [ ]:
df_p_2[(df_p_2.diferencia) == max(df_p_2.diferencia)]
# Mayor diferencia entre percentiles 60 y 40

category  subcategory  ...         price  diferencia
                                 ... percentile_75            
53451  Motorcycles  Motorcycles  ...    374.800655  194.800655

[1 rows x 14 columns]

In [ ]:
df_p_2[df_p_2.diferencia > 0].sort_values(by = 'diferencia', ascending = True).head(2)

category subcategory  ...         price diferencia
                                            ... percentile_75           
65435                   Phones       Meizu  ...       117.000     0.0048
69485  Repair and construction    Ceilings  ...         0.654     0.0048

[2 rows x 14 columns]

In [ ]:
print('Existen ', len(df_p_2[df_p_2.diferencia == 0]), ' combinaciones con una diferencia interpercentílica de 0€, lo cual representa un ', 
      round(len(df_p_2[df_p_2.diferencia == 0])/len(df_p_2)*100, 2), '% de los posibles grupos.', sep = '')

Existen 263 combinaciones con una diferencia interpercentílica de 0€, lo cual representa un 8.74% de los posibles grupos.


In [ ]:
df_p_2[(df_p_2.category == 'Appliances') & (df_p_2.subcategory == 'Climatic equipment') & (df_p_2.region == 'Altai region')]

category         subcategory  ...         price              
                                      ... percentile_70 percentile_75
1233  Appliances  Climatic equipment  ...          12.0          12.0

[1 rows x 13 columns]

El procedimiento para mostrar estos intervalos de precio será el siguiente:
* El usuario como input nos da categoría, subcategoría y región, a través del fichero descargado `'precios_subc.csv'`, le mostraremos por pantalla el intervalo de precios en el que se encuentra su subcategoría de producto para la región escogida.
> * `category = 'Appliances'`
> * `subcategory = 'Climatic equipment'`
> * `region = 'Altai region'`
>> Para articulos que se venden (`sold = 1`):
>> * `Intervalo de precios (50): [5.1€, 12€]`
>> * `Intervalo de precios (40): [6.36€, 12€]`
>> * `Intervalo de precios (20): [9.12€, 12€]`
>>> *Nosotros decidiremos como de ajustado queremos el intervalo de precios*
> * Si el precio se encuentra fuera del intervalo, al usuario se le advertirá que el precio es alto/bajo.


In [ ]:
from google.colab import files

In [ ]:
# Descargamos el fichero de precios por categoría, sucategoría y región
df_p_2.to_csv('precios_subc.csv')
files.download('precios_subc.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>